In [3]:
import numpy as np
import pandas as pd
import sklearn
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import rmsprop
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import utils
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [4]:
da=pd.read_csv('compresive_strength_concrete.csv')
da.head()
da=da.iloc[np.random.permutation(len(da))] #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
                                                           # see upper link                
# ss = sklearn.utils.shuffle(sonar)    
da


,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
667,192.0,288.0,0.0,192.0,0.0,929.8,716.1,28,39.32
933,167.0,187.0,195.0,185.0,7.0,898.0,636.0,28,23.89
364,214.9,53.8,121.9,155.6,9.6,1014.3,780.6,3,18.02
11,198.6,132.4,0.0,192.0,0.0,978.4,825.5,28,28.02
128,401.8,94.7,0.0,147.4,11.4,946.8,852.1,28,68.50
...,...,...,...,...,...,...,...,...,...
111,362.6,189.0,0.0,164.9,11.6,944.7,755.8,7,55.90
442,213.5,0.0,174.2,159.2,11.7,1043.6,771.9,28,44.64
164,425.0,106.3,0.0,153.5,16.5,852.1,887.1,91,65.20
272,231.8,0.0,121.6,174.0,6.7,1056.4,778.5,56,42.70


In [5]:
 train, validate, test = np.split(da.sample(frac=1), [int(.6*len(da)), int(0.9*len(da))])
# print(train.shape)
# print(validate.shape)
# print(test.shape)
# print(da.shape)
X_train = train.iloc[:,0:-1].values
y_train= train.iloc[:,-1].values
# encoder =  LabelEncoder()
# y_train= encoder.fit_transform(y_train)
# Y_train = pd.get_dummies(y_train).values
##########################
X_test = test.iloc[:,0:-1].values
y_test= test.iloc[:,-1].values
# encoder =  LabelEncoder()
# y_test= encoder.fit_transform(y_test)
# Y_test = pd.get_dummies(y_test).values
##########################
X_validate = validate.iloc[:,0:-1].values
y_validate= validate.iloc[:,-1].values
# encoder =  LabelEncoder()
# y_validate= encoder.fit_transform(y_validate)
# Y_validate = pd.get_dummies(y_validate).values
print(X_train.shape)
print(X_test.shape)
print(X_validate.shape)




(618, 8)
(103, 8)
(309, 8)


In [6]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
    input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model
# model = Sequential()
# model.add(Dense(8,input_shape=(8,), activation='relu') )
# model.add(Dense(8,activation='relu')) 
# model.add(Dense(1)) 
# model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])

# k-fold validation

In [7]:
traink,testk= np.split(da.sample(frac=1), [int(0.9*len(da))])
train_data=traink.iloc[:,0:-1].values
train_targets=traink.iloc[:,-1].values
print(train_data.shape)
print(train_targets.shape)
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)


    

(927, 8)
(927,)
processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [10]:
print(all_scores)
np.mean(all_scores)

[4.274587154388428, 5.197977066040039, 4.953444004058838, 5.175831317901611]


4.900459885597229

# model

In [11]:
history=model.fit(X_train, y_train, epochs=10,batch_size=1,validation_data=(X_validate,y_validate))


Train on 618 samples, validate on 309 samples
Epoch 1/10
618/618 [==============================] - 1s 2ms/step - loss: 40.4391 - mae: 4.7876 - val_loss: 33.0348 - val_mae: 4.2957
Epoch 2/10
618/618 [==============================] - 1s 2ms/step - loss: 38.3967 - mae: 4.6549 - val_loss: 42.3339 - val_mae: 5.1326
Epoch 3/10
618/618 [==============================] - 1s 2ms/step - loss: 36.7577 - mae: 4.5586 - val_loss: 43.8659 - val_mae: 5.2125
Epoch 4/10
618/618 [==============================] - 1s 2ms/step - loss: 38.4222 - mae: 4.7213 - val_loss: 37.0527 - val_mae: 4.4339
Epoch 5/10
618/618 [==============================] - 1s 2ms/step - loss: 35.6731 - mae: 4.5199 - val_loss: 38.8928 - val_mae: 4.7873
Epoch 6/10
618/618 [==============================] - 1s 2ms/step - loss: 36.7102 - mae: 4.5911 - val_loss: 42.9324 - val_mae: 4.8694
Epoch 7/10
618/618 [==============================] - 2s 2ms/step - loss: 37.2984 - mae: 4.6244 - val_loss: 36.5767 - val_mae: 4.4444
Epoch 8/10
618/6

In [12]:
test_loss, test_mae = model.evaluate(X_test,y_test)
print('test_mae:', test_mae)
print('test_loss:',test_loss)
mae = history.history['mae']
val_mae = history.history['val_mae']
print('train_mae:', np.mean(mae) )
print('val_mae:',np.mean(val_mae))
history_dict=history.history
# DF=pd.DataFrame(history_dict)
# DF
#export_csv = DF.to_csv (r'C:\Users\Muhammad hamza\Desktop\export_dataframe.csv', index = None, header=True)

103/103 [==============================] - 0s 68us/step
test_mae: 4.267749786376953
test_loss: 30.437123474565524
train_mae: 4.624419
val_mae: 4.773802900314331


In [1]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.clf()

NameError: name 'history' is not defined

In [2]:
mae = history.history['mae']
val_mae = history.history['val_mae']
plt.plot(epochs, mae, 'bo', label='Training mae')
plt.plot(epochs, val_mae, 'b', label='Validation mae')
plt.title('Training and validation mae')
plt.xlabel('Epochs')
plt.ylabel('mae')
plt.legend()
plt.show()

NameError: name 'history' is not defined